# Randomised Iterative Improvement on MAXSAT



Set classpath to the pre-compiled jar

In [ ]:
@file:DependsOn("../build/libs/kglsm.jar")

Define imports

In [ ]:
import com.sihvi.glsm.problem.MAXSAT
import com.sihvi.glsm.sls.GLSMBuilder
import com.sihvi.glsm.sls.StateMachineTransition
import com.sihvi.glsm.space.BooleanSearchSpace
import com.sihvi.glsm.space.DiscreteSearchSpace
import com.sihvi.glsm.strategy.IIMode
import com.sihvi.glsm.strategy.IterativeImprovementStrategy
import com.sihvi.glsm.strategy.RandomWalkStrategy
import com.sihvi.glsm.transitionpredicate.NoImprovementPredicate
import com.sihvi.glsm.transitionpredicate.NotPredicate
import com.sihvi.glsm.transitionpredicate.ProbabilisticPredicate
import com.sihvi.glsm.memory.Memory
import com.sihvi.glsm.memory.BasicMemory
import com.sihvi.glsm.memory.BasicSolution

## Problem

We are using MAXSAT problem as an example. Get an instance of the problem.

In [ ]:
val noVariables = 20

val problemInstance = MAXSAT.getRandomInstance(noVariables,50,2)
println(problemInstance)

## Search space
Boolean search space defines two methods on an array of booleans: to get a neighbourhood and a random neighbour.

In this case, the members of the neighbourhood are exactly Hamming distance of 1 away from the current solution (i.e. one boolean value flipped) 

In [ ]:
val space = BooleanSearchSpace(noVariables)

## Memory
Basic memory that holds:
* Current solution and its cost
* Best solution and best cost
* Number of steps performed
* Number of steps performed without improvement (when it was expected)

In [ ]:
val initialSolution = space.getInitial()
val memory = BasicMemory(BasicSolution(initialSolution, problemInstance.evaluate(initialSolution)))

## GLSM and Strategies
Randomised Iterative Improvement consists of two strategies that are flipped probabilistically
* Iterative Best Improvement Strategy -- picks a solution from a neighbourhood that gives the best improvement
* Random Walk Strategy -- randomly picks a solution from a neighbourhood

The termination predicate of choice here is No Improvement Predicate, i.e. we terminate the search if there were n steps without improvement (in this case 10)

In [ ]:
val terminationPredicate = NoImprovementPredicate(10)

val wp = 0.1
val toRandomPredicate = ProbabilisticPredicate(to = wp)
val toIIPredicate = NotPredicate(toRandomPredicate)

val walk = RandomWalkStrategy<Boolean>()
val iterativeImprovement = IterativeImprovementStrategy<Boolean>(IIMode.BEST, true)

As all the components are defined, we can now build the GLSM with strategies and transitions between them 

In [ ]:
val glsm = GLSMBuilder<Boolean, BasicSolution<Boolean>, Memory<Boolean, BasicSolution<Boolean>>, DiscreteSearchSpace<Boolean>>()
        .addStrategy(iterativeImprovement)
        .addStrategy(walk)
        .addTransition(StateMachineTransition(0, 1, toRandomPredicate))
        .addTransition(StateMachineTransition(1, 0, toIIPredicate))
        .addTransition(StateMachineTransition(0, -1, terminationPredicate))
        .build()

# Solve

With everything ready we can run GLSM on our problem instance

In [ ]:
val finalSolution = glsm.solve(memory, space, problemInstance)

println("Steps taken: " + memory.stepCount)
println("Solution: " + finalSolution.solution.joinToString(", "))
println("Cost: " + finalSolution.cost)